In [1]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

# Tutorial 6  
Graph AutoEncoders GAE &  
Variational Graph Autoencoders VGAE    

[paper](https://arxiv.org/pdf/1611.07308.pdf)  
[code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py)

## Graph AutoEncoder GAE

### Load the data

In [2]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Data(edge_index=[2, 9104], test_mask=[3327], train_mask=[3327], val_mask=[3327], x=[3327, 3703], y=[3327])

In [3]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(edge_index=[2, 9104], x=[3327, 3703], y=[3327])

In [4]:
data = train_test_split_edges(data)

In [5]:
data

Data(test_neg_edge_index=[2, 455], test_pos_edge_index=[2, 455], train_neg_adj_mask=[3327, 3327], train_pos_edge_index=[2, 7740], val_neg_edge_index=[2, 227], val_pos_edge_index=[2, 227], x=[3327, 3703], y=[3327])

### Define the Encoder

In [6]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

### Define the Autoencoder

In [7]:
from torch_geometric.nn import GAE

In [8]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)



In [10]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6145, AP: 0.6514
Epoch: 002, AUC: 0.6269, AP: 0.6642
Epoch: 003, AUC: 0.6315, AP: 0.6691
Epoch: 004, AUC: 0.6334, AP: 0.6713
Epoch: 005, AUC: 0.6346, AP: 0.6726
Epoch: 006, AUC: 0.6351, AP: 0.6741
Epoch: 007, AUC: 0.6359, AP: 0.6752
Epoch: 008, AUC: 0.6365, AP: 0.6764
Epoch: 009, AUC: 0.6372, AP: 0.6779
Epoch: 010, AUC: 0.6379, AP: 0.6802
Epoch: 011, AUC: 0.6389, AP: 0.6824
Epoch: 012, AUC: 0.6399, AP: 0.6850
Epoch: 013, AUC: 0.6406, AP: 0.6876
Epoch: 014, AUC: 0.6416, AP: 0.6907
Epoch: 015, AUC: 0.6421, AP: 0.6941
Epoch: 016, AUC: 0.6426, AP: 0.6973
Epoch: 017, AUC: 0.6430, AP: 0.7002
Epoch: 018, AUC: 0.6432, AP: 0.7035
Epoch: 019, AUC: 0.6434, AP: 0.7059
Epoch: 020, AUC: 0.6431, AP: 0.7074
Epoch: 021, AUC: 0.6431, AP: 0.7091
Epoch: 022, AUC: 0.6433, AP: 0.7105
Epoch: 023, AUC: 0.6435, AP: 0.7114
Epoch: 024, AUC: 0.6437, AP: 0.7121
Epoch: 025, AUC: 0.6441, AP: 0.7128
Epoch: 026, AUC: 0.6446, AP: 0.7142
Epoch: 027, AUC: 0.6448, AP: 0.7147
Epoch: 028, AUC: 0.6456, AP:

In [11]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[ 0.4781, -0.6356],
        [-0.9415,  1.1525],
        [ 0.5803, -0.7327],
        ...,
        [-0.3368,  0.3879],
        [ 0.5803, -0.7327],
        [ 0.5803, -0.7327]], device='cuda:0', grad_fn=<AddBackward0>)

## Are the results (AUC) and (AP) easy to read and compare?

# Use Tensorboard

In [12]:
from torch.utils.tensorboard import SummaryWriter

In [13]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Import tensorboard

#### Installation: (if needed) "pip install tensorboard"

In [14]:
writer = SummaryWriter('runs/GAE1_experiment_'+'2d_100_epochs')

In [15]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.5902, AP: 0.6221
Epoch: 002, AUC: 0.6100, AP: 0.6425
Epoch: 003, AUC: 0.6175, AP: 0.6520
Epoch: 004, AUC: 0.6227, AP: 0.6574
Epoch: 005, AUC: 0.6269, AP: 0.6621
Epoch: 006, AUC: 0.6294, AP: 0.6647
Epoch: 007, AUC: 0.6310, AP: 0.6672
Epoch: 008, AUC: 0.6326, AP: 0.6700
Epoch: 009, AUC: 0.6342, AP: 0.6729
Epoch: 010, AUC: 0.6359, AP: 0.6770
Epoch: 011, AUC: 0.6372, AP: 0.6806
Epoch: 012, AUC: 0.6389, AP: 0.6853
Epoch: 013, AUC: 0.6401, AP: 0.6896
Epoch: 014, AUC: 0.6415, AP: 0.6938
Epoch: 015, AUC: 0.6420, AP: 0.6975
Epoch: 016, AUC: 0.6420, AP: 0.7015
Epoch: 017, AUC: 0.6427, AP: 0.7048
Epoch: 018, AUC: 0.6425, AP: 0.7065
Epoch: 019, AUC: 0.6425, AP: 0.7080
Epoch: 020, AUC: 0.6432, AP: 0.7096
Epoch: 021, AUC: 0.6434, AP: 0.7109
Epoch: 022, AUC: 0.6439, AP: 0.7118
Epoch: 023, AUC: 0.6443, AP: 0.7128
Epoch: 024, AUC: 0.6452, AP: 0.7139
Epoch: 025, AUC: 0.6459, AP: 0.7150
Epoch: 026, AUC: 0.6469, AP: 0.7162
Epoch: 027, AUC: 0.6485, AP: 0.7180
Epoch: 028, AUC: 0.6504, AP:

## Graph Variational AutoEncoder (GVAE)

In [21]:
from torch_geometric.nn import VGAE

In [22]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [26]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [27]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [28]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6165, AP: 0.6573
Epoch: 002, AUC: 0.6305, AP: 0.6628
Epoch: 003, AUC: 0.6340, AP: 0.6635
Epoch: 004, AUC: 0.6355, AP: 0.6642
Epoch: 005, AUC: 0.6364, AP: 0.6645
Epoch: 006, AUC: 0.6373, AP: 0.6656
Epoch: 007, AUC: 0.6374, AP: 0.6662
Epoch: 008, AUC: 0.6374, AP: 0.6664
Epoch: 009, AUC: 0.6375, AP: 0.6669
Epoch: 010, AUC: 0.6377, AP: 0.6674
Epoch: 011, AUC: 0.6379, AP: 0.6677
Epoch: 012, AUC: 0.6380, AP: 0.6679
Epoch: 013, AUC: 0.6380, AP: 0.6683
Epoch: 014, AUC: 0.6382, AP: 0.6687
Epoch: 015, AUC: 0.6385, AP: 0.6690
Epoch: 016, AUC: 0.6386, AP: 0.6694
Epoch: 017, AUC: 0.6388, AP: 0.6700
Epoch: 018, AUC: 0.6391, AP: 0.6705
Epoch: 019, AUC: 0.6392, AP: 0.6709
Epoch: 020, AUC: 0.6394, AP: 0.6713
Epoch: 021, AUC: 0.6394, AP: 0.6715
Epoch: 022, AUC: 0.6395, AP: 0.6718
Epoch: 023, AUC: 0.6394, AP: 0.6721
Epoch: 024, AUC: 0.6392, AP: 0.6723
Epoch: 025, AUC: 0.6391, AP: 0.6724
Epoch: 026, AUC: 0.6391, AP: 0.6727
Epoch: 027, AUC: 0.6391, AP: 0.6729
Epoch: 028, AUC: 0.6389, AP:

Epoch: 230, AUC: 0.7920, AP: 0.7981
Epoch: 231, AUC: 0.7926, AP: 0.7983
Epoch: 232, AUC: 0.7937, AP: 0.7986
Epoch: 233, AUC: 0.7945, AP: 0.7985
Epoch: 234, AUC: 0.7944, AP: 0.7984
Epoch: 235, AUC: 0.7935, AP: 0.7983
Epoch: 236, AUC: 0.7929, AP: 0.7985
Epoch: 237, AUC: 0.7921, AP: 0.7980
Epoch: 238, AUC: 0.7927, AP: 0.7983
Epoch: 239, AUC: 0.7930, AP: 0.7979
Epoch: 240, AUC: 0.7929, AP: 0.7978
Epoch: 241, AUC: 0.7923, AP: 0.7978
Epoch: 242, AUC: 0.7913, AP: 0.7977
Epoch: 243, AUC: 0.7910, AP: 0.7975
Epoch: 244, AUC: 0.7913, AP: 0.7973
Epoch: 245, AUC: 0.7918, AP: 0.7971
Epoch: 246, AUC: 0.7922, AP: 0.7971
Epoch: 247, AUC: 0.7918, AP: 0.7969
Epoch: 248, AUC: 0.7911, AP: 0.7969
Epoch: 249, AUC: 0.7902, AP: 0.7965
Epoch: 250, AUC: 0.7901, AP: 0.7966
Epoch: 251, AUC: 0.7905, AP: 0.7965
Epoch: 252, AUC: 0.7913, AP: 0.7966
Epoch: 253, AUC: 0.7916, AP: 0.7964
Epoch: 254, AUC: 0.7912, AP: 0.7962
Epoch: 255, AUC: 0.7904, AP: 0.7961
Epoch: 256, AUC: 0.7889, AP: 0.7957
Epoch: 257, AUC: 0.7885, AP: